# 1.Import Libraries

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import joblib

# 2. Load Data

In [14]:
# Load the data
data = pd.read_csv('nuts.csv')

# Labels
# 0 -> M3
# 1 -> M4
# 2 -> M5

# Assuming the last column is the label
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Saving Scaler
joblib.dump(scaler, 'scaler.pkl')

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Training the data

In [15]:
# Define the neural network architecture
class NutClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NutClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Determine the number of unique classes
num_classes = len(np.unique(y))

# Instantiate the model with the correct number of input features and classes
model = NutClassifier(input_size=X_train.shape[1], num_classes=num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')

Epoch 1/20, Loss: 1.0955249071121216
Epoch 2/20, Loss: 1.0769516229629517
Epoch 3/20, Loss: 1.0590200424194336
Epoch 4/20, Loss: 1.0415866374969482
Epoch 5/20, Loss: 1.0245182514190674
Epoch 6/20, Loss: 1.0079649686813354
Epoch 7/20, Loss: 0.9918846487998962
Epoch 8/20, Loss: 0.976055383682251
Epoch 9/20, Loss: 0.960517406463623
Epoch 10/20, Loss: 0.9451580047607422
Epoch 11/20, Loss: 0.9297683835029602
Epoch 12/20, Loss: 0.9146416783332825
Epoch 13/20, Loss: 0.8998124599456787
Epoch 14/20, Loss: 0.8850808143615723
Epoch 15/20, Loss: 0.8704819679260254
Epoch 16/20, Loss: 0.8560521006584167
Epoch 17/20, Loss: 0.8417966365814209
Epoch 18/20, Loss: 0.8277502655982971
Epoch 19/20, Loss: 0.8140113949775696
Epoch 20/20, Loss: 0.8003965020179749
Accuracy on test set: 50.0%


# 4. Save Model

In [16]:
# Save the model weights
torch.save(model.state_dict(), 'nut_classifier_weights.pth')

# 5. Example Usage

In [12]:
import torch
import torch.nn as nn

# Define the neural network architecture (same as the one used during training)
class NutClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NutClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model with the correct input size and number of classes
input_size = 6  # Replace with the actual number of input features
num_classes = 3  # Replace with the actual number of classes
model = NutClassifier(input_size=input_size, num_classes=num_classes)

# Load the model weights
model.load_state_dict(torch.load('nut_classifier_weights.pth'))
model.eval()  # Set the model to evaluation mode

# Example usage: making predictions
# Replace `example_input` with actual input data
example_input = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6]], dtype=torch.float32)
output = model(example_input)
_, predicted_class = torch.max(output.data, 1)
print(f'Predicted class: {predicted_class.item()}')

Predicted class: 2


C:\Users\K0NQ\AppData\Local\Temp\ipykernel_14084\288749272.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('nut_classifier_weights.pth'

In [20]:
import joblib
import numpy as np
import torch

# Load the scaler
scaler = joblib.load('scaler.pkl')

# Example input data (replace with actual input data)
input_2 = [210, 1669.0, 182.0, 1452.5, 134.5, 1057.0]
example_input = np.array([input_2])

# Standardize the input data
example_input_standardized = scaler.transform(example_input)

# Convert the standardized input data to a PyTorch tensor
example_input_tensor = torch.tensor(example_input_standardized, dtype=torch.float32)

# Load the model weights (assuming the model architecture is defined as in the previous code)
model.load_state_dict(torch.load('nut_classifier_weights.pth'))
model.eval()  # Set the model to evaluation mode

# Make predictions using the loaded model
output = model(example_input_tensor)
_, predicted_class = torch.max(output.data, 1)
print(f'Predicted class: {predicted_class.item()}')

Predicted class: 2


C:\Users\K0NQ\AppData\Local\Temp\ipykernel_14084\2116452981.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('nut_classifier_weights.pth